Chủ đề T12: **Sàng lọc thuốc tác động hệ thần kinh trung ương (CNS)** là một hướng nghiên cứu quan trọng trong phát triển thuốc, đặc biệt liên quan đến các bệnh như Alzheimer, Parkinson, động kinh, trầm cảm, và loạn thần. Dưới đây là phần phân tích song ngữ, hướng dẫn đầy đủ theo chuẩn AIMLOps template.

---

## 🧠 T12: CNS-Targeted Drug Screening using ChEMBL and RDKit

### 1. 🎯 Phân tích nội dung chủ đề | Topic Objective Analysis

**Việt:**
Chủ đề này tập trung vào việc khai thác dữ liệu từ ChEMBL 35 để trích xuất các phân tử có hoạt tính sinh học trên các target của hệ thần kinh trung ương (CNS targets). Mục tiêu là tìm ra các hợp chất có tiềm năng trở thành thuốc điều trị các bệnh thần kinh.

**English:**
This topic focuses on mining ChEMBL 35 to extract molecules with biological activity against central nervous system (CNS) targets. The goal is to identify compounds with potential as drug candidates for neurological disorders.

---

### 2. 🔬 Mô hình phân tích | Analytical Workflow

1. **Truy vấn dữ liệu** từ PostgreSQL/ChEMBL để lọc các phân tử có hoạt tính trên các CNS targets.
2. **Tiền xử lý dữ liệu**: lọc IC50 hợp lệ, loại bỏ giá trị NULL, convert đơn vị nếu cần.
3. **Tính descriptor với RDKit**: LogP, MolWt, TPSA, HBA, HBD.
4. **Áp dụng các tiêu chí sàng lọc CNS** như:

   * MolWt < 450
   * TPSA < 90 Å²
   * LogP từ 1 đến 4
5. **Trực quan hóa** kết quả bằng matplotlib hoặc seaborn.
6. **Lưu kết quả ra `../data/` và code trong `notebook/`**.

---

### 3. 💡 Hướng dẫn song ngữ | Bilingual Instructions

#### Step 1: SQL query for CNS target molecules

**Việt:** Truy vấn các phân tử có hoạt tính IC50 trên mục tiêu thuộc hệ thần kinh trung ương.
**English:** Query molecules with IC50 activity on CNS-related targets.

✅ File lưu: `t12_cns_targets.sql` → Kết quả: `../data/t12_cns_targets.csv`

```sql
-- t12_cns_targets.sql
SELECT DISTINCT
    md.chembl_id,
    cs.canonical_smiles,
    act.standard_value::float AS ic50_nm,
    act.standard_type,
    td.pref_name AS target_name
FROM activities act
JOIN compound_structures cs ON act.molregno = cs.molregno
JOIN molecule_dictionary md ON cs.molregno = md.molregno
JOIN target_dictionary td ON act.tid = td.tid
WHERE act.standard_type = 'IC50'
  AND act.standard_value ~ '^[0-9\.]+$'
  AND td.pref_name ILIKE '%central nervous system%'
  LIMIT 100;
```

---

#### Step 2: Python to load and filter molecules by CNS properties

```python
# t12_filter_cns_drugs.py
import pandas as pd
import os
import warnings
from rdkit import Chem
from rdkit.Chem import Descriptors

warnings.filterwarnings("ignore", category=UserWarning)

base_path = ".."
data_path = os.path.join(base_path, "data", "t12_cns_targets.csv")
df = pd.read_csv(data_path)

df['mol'] = df['canonical_smiles'].apply(Chem.MolFromSmiles)
df['MolWt'] = df['mol'].apply(Descriptors.MolWt)
df['TPSA'] = df['mol'].apply(Descriptors.TPSA)
df['LogP'] = df['mol'].apply(Descriptors.MolLogP)

# CNS criteria filter
df_cns = df[(df['MolWt'] < 450) & (df['TPSA'] < 90) & (df['LogP'].between(1, 4))]

output_path = os.path.join(base_path, "data", "t12_cns_filtered.csv")
df_cns.to_csv(output_path, index=False)
print(f"Filtered CNS candidates saved to {output_path}")
```

---

### 4. 💻 5 ví dụ thực tế | Practical SQL + Python Use Cases

#### 4.1. Truy vấn target liên quan đến Alzheimer

**SQL**: `t12_alzheimer.sql`
**Python**: tính MolWt, TPSA, LogP
→ file: `t12_alzheimer.csv`

#### 4.2. Trích thuốc ức chế MAO (Monoamine oxidase)

**SQL**: `t12_mao_inhibitors.sql`
→ dùng `%monoamine oxidase%`
→ lọc các phân tử < 500 Da

#### 4.3. Truy vấn phân tử ảnh hưởng GABA receptor

**SQL**: `t12_gaba_targets.sql`
→ `%GABA%`
→ lọc LogP từ 2 đến 5

#### 4.4. Tính toán descriptor và lọc theo các tiêu chí BBB (Blood-Brain Barrier)

**Python**: TPSA < 90, LogP từ 2–4 → lưu `t12_bbb_filtered.csv`

#### 4.5. So sánh phân bố descriptor giữa tất cả phân tử vs phân tử đạt CNS tiêu chí

**Python**: overlay histogram (MolWt, TPSA)

---

Nếu bạn muốn, tôi có thể tạo ngay:

* ✅ 5 file `.sql` mẫu
* ✅ 5 file `.py` mẫu
* ✅ Jupyter Notebook `notebook/T12_1_cns_filtering.ipynb`
* ✅ Kèm theo sơ đồ workflow CNS drug filtering (SVG hoặc PNG)

Bạn có muốn tôi tạo luôn không?


In [1]:
# t12_filter_cns_drugs.py
import pandas as pd
import os
import warnings
from rdkit import Chem
from rdkit.Chem import Descriptors

warnings.filterwarnings("ignore", category=UserWarning)

base_path = ".."
data_path = os.path.join(base_path, "data", "t12_cns_targets.csv")
df = pd.read_csv(data_path)

df['mol'] = df['canonical_smiles'].apply(Chem.MolFromSmiles)
df['MolWt'] = df['mol'].apply(Descriptors.MolWt)
df['TPSA'] = df['mol'].apply(Descriptors.TPSA)
df['LogP'] = df['mol'].apply(Descriptors.MolLogP)

# CNS criteria filter
df_cns = df[(df['MolWt'] < 450) & (df['TPSA'] < 90) & (df['LogP'].between(1, 4))]

output_path = os.path.join(base_path, "data", "t12_cns_filtered.csv")
df_cns.to_csv(output_path, index=False)
print(f"Filtered CNS candidates saved to {output_path}")


Filtered CNS candidates saved to ..\data\t12_cns_filtered.csv


Tiếp theo, ta sẽ đi sâu vào từng phần trong quy trình sàng lọc thuốc tác động hệ thần kinh trung ương (CNS drug screening) để làm rõ **cơ sở khoa học**, **ưu điểm**, **ứng dụng thực tế**, và **các cải tiến có thể áp dụng** cho chủ đề **T12**.

---

## 🔍 5. Phân tích sâu quy trình sàng lọc CNS-targeted drugs

### 🧬 1. CNS Target Selection (Lựa chọn mục tiêu CNS)

**Việt:** Các mục tiêu sinh học trong hệ thần kinh trung ương bao gồm các thụ thể thần kinh (GABA, NMDA, 5-HT), enzym (MAO-A, AChE), và protein liên quan đến các bệnh như Alzheimer, Parkinson, trầm cảm, loạn thần…

**English:** CNS targets include neurotransmitter receptors (e.g., GABA, NMDA, 5-HT), enzymes (MAO-A, AChE), and proteins implicated in diseases like Alzheimer’s, Parkinson’s, depression, and psychosis.

🔑 **Lưu ý:**

* Có thể truy vấn theo `pref_name` hoặc `target_type` trong bảng `target_dictionary`.
* Nên dùng `ILIKE` để tìm kiếm không phân biệt chữ hoa/thường.

---

### 🔎 2. Activity Value Filtering (Lọc dữ liệu hoạt tính)

**Việt:** Nên lọc các giá trị IC50 hợp lệ dạng số, tránh NULL hoặc chuỗi mô tả (">10 µM").

**English:** Filter valid numeric IC50 values to avoid nulls or strings like ">10 µM".

✅ **Regex** dùng: `~ '^[0-9\.]+$'`
💡 Tránh lỗi: `operator does not exist: numeric ~ unknown` → ép kiểu: `act.standard_value::float`

---

### ⚗️ 3. Descriptor Calculation with RDKit

Các thông số được tính từ SMILES:

| Descriptor | Mô tả tiếng Việt   | Ý nghĩa dược động học       |
| ---------- | ------------------ | --------------------------- |
| `MolWt`    | Khối lượng phân tử | ảnh hưởng độ hấp thu        |
| `TPSA`     | Diện tích phân cực | liên quan BBB & hấp thu     |
| `LogP`     | Thân dầu nước      | ảnh hưởng độ thấm màng      |
| `HBD/HBA`  | Nhóm cho/nhận H+   | ảnh hưởng kết dính receptor |

🎯 Mục tiêu cho CNS:

* `MolWt < 450`
* `TPSA < 90`
* `1 < LogP < 4`
* `HBA ≤ 8`, `HBD ≤ 2`

---

### 📊 4. Visualization & Analysis

**Các dạng biểu đồ gợi ý:**

* 📈 Histogram: Phân bố LogP, TPSA
* 🎯 Violin plot: So sánh phân bố LogP giữa “pass CNS filter” vs “fail”
* 📉 PCA plot: biểu diễn không gian hóa học

---

### 🧪 5. Áp dụng vào Drug Discovery Pipeline

**Ứng dụng thực tế:**

* Sàng lọc nhanh các thư viện ảo (virtual libraries)
* Phân tích các fragment CNS-like
* Lọc compound trước docking/MD simulation

**Mở rộng tiềm năng:**

* Kết hợp mô hình ML (QSAR, classification)
* Dự đoán khả năng vượt BBB bằng mô hình đã huấn luyện

---

## 🛠 6. Gợi ý tên file theo chuẩn AIMLOps

| Mô tả                      | Tên file SQL             | Tên file Python                       | Tên output CSV          |
| -------------------------- | ------------------------ | ------------------------------------- | ----------------------- |
| Trích IC50 cho CNS targets | `t12_cns_targets.sql`    | —                                     | `t12_cns_targets.csv`   |
| Tính descriptor & lọc      | —                        | `t12_filter_cns_drugs.py`             | `t12_cns_filtered.csv`  |
| Alzheimer targets          | `t12_alzheimer.sql`      | `t12_alzheimer_filter.py`             | `t12_alzheimer.csv`     |
| MAO inhibitors             | `t12_mao_inhibitors.sql` | —                                     | `t12_mao_filtered.csv`  |
| GABA receptor              | `t12_gaba_targets.sql`   | `t12_gaba_filter.py`                  | `t12_gaba_filtered.csv` |
| Visualization              | —                        | `t12_plot_descriptor_distribution.py` | —                       |

---

## 🧠 7. Đề xuất nâng cao (nếu có GPU/máy mạnh)

1. 🧪 **Dùng AutoML** (H2O.ai, Auto-sklearn) để học mô hình phân biệt CNS-like và non-CNS-like compounds.
2. ⚛️ **Chạy Mol2Vec hoặc ECFP fingerprints** và áp dụng t-SNE cho mapping chemical space.
3. 📊 **Tạo Streamlit dashboard** tương tác cho hiển thị và lọc CNS-like molecules.

---

Nếu bạn muốn, tôi sẽ tạo luôn:

* ✅ Các file `.sql`, `.py`, `.ipynb` tương ứng với T12\_1
* ✅ Biểu đồ phân tích
* ✅ File markdown `T12_CNS_drug_filtering.md` tổng hợp

📌 Bạn muốn tôi bắt đầu từ phần nào tiếp theo? Hay tạo toàn bộ cấu trúc code + dữ liệu mẫu T12?
